In [ ]:
import os
if os.path.basename(os.getcwd())=='experiments':
    os.chdir("./..")

In [ ]:
import wandb
from model.hierarchical_size_model_new import *
from utils.data_preparation import *
from utils.evaluation import *
from utils.experiment import default_config

In [ ]:
def no_training_test(config, notes, project="clothing-fit", group="no_training", entity="clothing-fit-licencjat"):
    run = wandb.init(project=project, config=config, notes=notes, group=group, entity=entity)
    if "data_info" not in wandb.config:
        data_info = {"train":{}, "test":{}}
        train = get_processed_renttherunway_data(data_info=data_info["train"])
        test = get_test_runttherunway_data(data_info=data_info["test"])
        wandb.config.update({"data_info": data_info})
    else:
        data_info = wandb.config.data_info
        train = get_data_from_config(data_info["train"])
        test = get_data_from_config(data_info["test"])
    model = HierarchicalSize(train, wandb.config.default_learning_rate, config=wandb.config)
    wandb.config.update(model.get_model_config())
    run.tags = run.tags + model.tags
    log_values = model.get_parameters_stats()
    results = model.predict(test)
    log_values.update(result_stats_size_model(results))
    log_values.update(categorical_mean_prob_stats_size_model(results))
    wandb.log(log_values, step=model.iterations)
    wandb.finish()

In [ ]:
no_training_test(config=default_config(), notes="statified dataset, sigma_0, mu_0 per customer")

In [ ]:
wandb.finish()